'''
Program to read the input data and process it for the transformer model
- Read the file and split into english and corresponding python code
- format the python code into one line
    - factor new line
    - factor tabs

Also build a formatter that converts the single line python code
back to proper syntax for python. This will used for visualization
and verifing the output. This will be used to evaluate the model.


Challenges in the input data

    - Messy start various options to segment the input data 
        #write ; #1 ; #  write
    - clean input file
        - removed # comments len < 10 & 20 after checking 
        - updated few manually
        - remove comments in the code ???
        # dd - remove pattern

Segment the input file
    identify marker - messy data
    extract segment
    extract english sentence within segment
    extract python code within python
    format python code to one line




'''

In [1]:
'''
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
'''

In [2]:
!pip install SpaCy

In [3]:
import os
import re

import pandas as pd
import spacy

In [4]:
print(os.listdir())

['clean_data.txt', 'data_processing-v1.ipynb', 'data_processing.ipynb', 'out', 'p_data.csv', 'english_python_data.txt', 'analysis_shortv3.txt', 'analysis_shortv2.txt', 'clean_in10.txt', 'analysis.txt', 'clean_in10_20.txt', '.ipynb_checkpoints', 'analysis_short10.txt', 'analysis_short.txt']


In [5]:
#out dataframe
out_df = pd.DataFrame(columns = ['src', 'python'])

In [6]:
input_file = 'clean_data.txt'
output_file = os.path.join('analysis_shortv3.txt')
# first print all lines starting with # with count
with open(output_file,'w') as out_file:
    eng_buf = ''
    py_buf = ''
    samples = 0
    with open(input_file) as in_file:
        for idx,line in enumerate(in_file):
            if line.startswith('#') and len(line) > 30:
                samples = samples + 1
                out_df.loc[len(out_df)] = [eng_buf,py_buf]
                py_buf = ''
                eng_buf = line
                #print(line)
                #out_file.write(line)
            else :
                py_buf = py_buf + line
                #out_file.write(line)
            #if samples > 10:
                #break
print(f" Count of # :{idx}")

 Count of # :42424


In [7]:
len(out_df['python'])

4428

In [8]:
# Replace tabs with 4 spaces
out_df['python']= out_df['python'].str.replace('\t', '    ')

In [9]:
# clean spaces 
# 3->4
# 7->8
# 11->12

reg3s_pat = re.compile(r'(:?\n)[\s]{3}([\w])')
reg7s_pat = re.compile(r'(:?\n)[\s]{7}([\w])')
reg11s_pat = re.compile(r'(:?\n)[\s]{11}([\w])')

def regex_clean(val):

    clean_py = reg3s_pat.sub(r'\1    \2', val)
    clean_py = reg7s_pat.sub(r'\1        \2', clean_py)
    clean_py = reg11s_pat.sub(r'\1            \2', clean_py)
    
    return clean_py

out_df['trg'] = out_df['python'].apply(regex_clean)

In [10]:
# functiont to test

reg_pat = re.compile(r':?\n[\s]{3}[\w]')
def regex_filter(val):
    if val:
        mo = re.search(reg_pat,val)
        if mo:
            return True
        else:
            return False
    else:
        return False

df_filt = out_df[out_df['trg'].apply(regex_filter)]
len(df_filt)

0

In [11]:
spacy_en = spacy.load('en_core_web_sm')

In [12]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [13]:

#tokenize_en(val)

In [14]:
#kw_dict

In [22]:
import re
import keyword
import spacy
from spacy.tokenizer import Tokenizer
import random

idx = random.randint(0,len(out_df))
print(idx)
val = out_df['trg'].iloc[idx]

kw_dict = {}
for kw in keyword.kwlist:
    kw_dict[kw]= [{"ORTH":kw}]

# learn 4, 8 12 spaces
special_tabs = ['\\n    ','\\n        ','\\n            ']
for tab in special_tabs:
    kw_dict[tab] = [{"ORTH":tab}]
#kw_dict    

special_cases = kw_dict
prefix_re = re.compile(r'''^[\[\("']''')
suffix_re = re.compile(r'''[\]\)"']$''')
infix_re = re.compile(r'''(==|>=|<=|!=|\(|\)|:|"|\[|\]|=|,|')''')
#infix_re = re.compile(r'''(==|>=|<=|!=|:|=|,)''')
simple_url_re = re.compile(r'''^https?://''')

def custom_tokenizer(nlp):
    return Tokenizer(nlp.vocab, rules=special_cases,
                                #prefix_search=prefix_re.search,
                                #suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer)
                                #url_match=simple_url_re.match)

nlp = spacy.load("en_core_web_sm")
nlp.tokenizer = custom_tokenizer(nlp)
doc = nlp(val)
print("new :",[t.text for t in doc]) # ['hello', '-', 'world.', ':)']
print("old :",tokenize_en(val))

1152
new : ['\n', 'def', 'findTargetSumWays', '(', 'nums', ',', 'S', ')', ':', '\n    ', 'count', '=', '0', '\n    ', 'def', 'calculate', '(', 'nums', ',', 'i', ',', 'sum', ',', 'S', ')', ':', '\n        ', 'nonlocal', 'count', '\n        ', 'if', 'i', '==', 'len', '(', 'nums', ')', ':', '\n            ', 'if', 'sum', '==', 'S', ':', '\n                ', 'count', '+', '=', '1', '\n        ', 'else', ':', '\n            ', 'calculate', '(', 'nums', ',', 'i+1', ',', 'sum+', 'nums', '[', 'i', ']', ',', 'S', ')', '\n            ', 'calculate', '(', 'nums', ',', 'i+1', ',', 'sum-', 'nums', '[', 'i', ']', ',', 'S', ')', '\n\n    ', 'calculate', '(', 'nums', ',', '0', ',', '0', ',', 'S', ')', '\n    ', 'return', 'count', '\n\n\n\n\n']
old : ['\n', 'def', 'findTargetSumWays(nums', ',', 'S', '):', '\n    ', 'count', '=', '0', '\n    ', 'def', 'calculate(nums', ',', 'i', ',', 'sum', ',', 'S', '):', '\n        ', 'nonlocal', 'count', '\n        ', 'if', 'i', '=', '=', 'len(nums', '):', '\n      

In [29]:
# Get all keywords
kw_dict = {}
for kw in keyword.kwlist:
    kw_dict[kw]= [{"ORTH":kw}]

# learn 4, 8 12 spaces
special_tabs = ['\\n    ','\\n        ','\\n            ']
for tab in special_tabs:
    kw_dict[tab] = [{"ORTH":tab}]
#kw_dict    

special_cases = kw_dict
#prefix_re = re.compile(r'''^[\[\("']''')
#suffix_re = re.compile(r'''[\]\)"']$''')
infix_re = re.compile(r'''(==|>=|<=|!=|\(|\)|:|"|\[|\]|=|,|')''')
#infix_re = re.compile(r'''(==|>=|<=|!=|:|=|,)''')


def python_tokenizer(nlp):
    return Tokenizer(nlp.vocab, rules=special_cases,
                                #prefix_search=prefix_re.search,
                                #suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer)
                                #url_match=simple_url_re.match)


py_custom = python_tokenizer(spacy_en)

In [30]:
def tokenize_py(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in py_custom(text)]

In [31]:
print("new :",tokenize_py(val))
print("old :",tokenize_en(val))

new : ['\n', 'def', 'findTargetSumWays', '(', 'nums', ',', 'S', ')', ':', '\n    ', 'count', '=', '0', '\n    ', 'def', 'calculate', '(', 'nums', ',', 'i', ',', 'sum', ',', 'S', ')', ':', '\n        ', 'nonlocal', 'count', '\n        ', 'if', 'i', '==', 'len', '(', 'nums', ')', ':', '\n            ', 'if', 'sum', '==', 'S', ':', '\n                ', 'count', '+', '=', '1', '\n        ', 'else', ':', '\n            ', 'calculate', '(', 'nums', ',', 'i+1', ',', 'sum+', 'nums', '[', 'i', ']', ',', 'S', ')', '\n            ', 'calculate', '(', 'nums', ',', 'i+1', ',', 'sum-', 'nums', '[', 'i', ']', ',', 'S', ')', '\n\n    ', 'calculate', '(', 'nums', ',', '0', ',', '0', ',', 'S', ')', '\n    ', 'return', 'count', '\n\n\n\n\n']
old : ['\n', 'def', 'findTargetSumWays(nums', ',', 'S', '):', '\n    ', 'count', '=', '0', '\n    ', 'def', 'calculate(nums', ',', 'i', ',', 'sum', ',', 'S', '):', '\n        ', 'nonlocal', 'count', '\n        ', 'if', 'i', '=', '=', 'len(nums', '):', '\n           

In [32]:
out_df.to_csv('data.csv')